# Week 0 Day 3 Solutions

## Part 1: Building & Training a CNN

In [5]:
import torch as t
import torchvision
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import PIL
from PIL import Image
import json
from pathlib import Path
from typing import Union, Tuple, Callable, Optional
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm_notebook
import utils
import cnn_modules as cm

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
 
testset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [7]:
trainset[0][0].shape

torch.Size([1, 28, 28])

In [15]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = cm.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu = cm.ReLU()
        self.maxpool = cm.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = cm.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.flatten = cm.Flatten(1)
        self.fc1 = cm.Linear(4096, 128)
        self.fc2 = cm.Linear(128, 10)

    def forward(self, x: t.Tensor) -> t.Tensor:
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)

        return out

model = ConvNet()
# print(model)

In [16]:
device = t.device('cuda:0' if t.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [24]:
epochs = 3
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

MODEL_FILENAME = "./w1d2_convnet_mnist.pt"
device = "cuda" if t.cuda.is_available() else "cpu"

def train_convnet(trainloader: DataLoader, epochs: int, loss_fn: Callable) -> list:
    '''
    Defines a ConvNet using our previous code, and trains it on the data in trainloader.
    '''

    model = ConvNet().to(device).train()
    optimizer = t.optim.Adam(model.parameters())
    loss_list = []
    accuracy_list = []

    for epoch in range(epochs):

        progress_bar = tqdm_notebook(trainloader)
        for (x, y) in progress_bar:

            x = x.to(device)
            y = y.to(device)

            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_list.append(loss.item())

            y_predictions = y_hat.argmax(1)
            corrects = (y_predictions==y)
            accuracy = corrects.int().sum() / corrects.shape[0]
            accuracy_list.append(accuracy.item())

            progress_bar.set_description(f"Epoch = {epoch}, Loss = {loss.item():.4f} Accuracy = {accuracy.item()}")

    print(f"Saving model to: {MODEL_FILENAME}")
    t.save(model, MODEL_FILENAME)
    return loss_list, accuracy_list

loss_list, accuracy_list = train_convnet(trainloader, epochs, loss_fn)

fig = px.line(y=loss_list, template="simple_white")
fig.update_layout(title="Cross entropy loss on MNIST", yaxis_range=[0, max(loss_list)])
fig.show()

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

Saving model to: ./w1d2_convnet_mnist.pt


In [25]:
fig = px.line(y=accuracy_list, template="simple_white")
fig.update_layout(title="Accuracy on MNIST", yaxis_range=[0, max(accuracy_list)])
fig.show()